# IPIP-NEO-120 Scores 🌊

Based on user responses we calculate scores in the Big-Five version [IPIP-NEO](https://ipip.ori.org/).

Official repository version:

- [https://osf.io/tbmh5](https://osf.io/tbmh5/)

About dataset:

- [https://osf.io/2kfhe](https://osf.io/2kfhe)

In [1]:
import pyreadstat

import pandas as pd
import traits as T

from ipipneo import IpipNeo

import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_columns = 10
pd.options.display.max_rows = 10

In [2]:
ORIGINAL_IPIP_NEO_DATASET = "./datasets/raw/IPIP120.por"
NQ_IPIP_NEO_DATASET = "./datasets/refined/IPIP120-SCORES.csv.gz"

## 1. Original dataset in SPSS portable file format 🗂️

In [3]:
df0, _ = pyreadstat.read_por(ORIGINAL_IPIP_NEO_DATASET)

dflen = len(df0)
assert dflen == 619150

df0.head(5)

,CASE,SEX,AGE,SEC,MIN,...,I116,I117,I118,I119,I120
0,1.0,2.0,19.0,8.0,41.0,...,2.0,4.0,1.0,5.0,4.0
1,2.0,2.0,22.0,24.0,45.0,...,4.0,2.0,3.0,4.0,3.0
2,4.0,2.0,22.0,3.0,57.0,...,2.0,4.0,2.0,5.0,4.0
3,5.0,2.0,22.0,44.0,4.0,...,1.0,5.0,3.0,5.0,3.0
4,6.0,1.0,13.0,14.0,6.0,...,2.0,4.0,4.0,3.0,5.0


## 2. Values ​​with zero in items are removed ❌ 

In [4]:
df0_float_cols = df0.select_dtypes(include=["float64"]).columns
df0[df0_float_cols] = df0[df0_float_cols].astype(int)
df0.columns = df0.columns.str.strip().str.lower()

print(f"Dataset original total: {dflen}")
print(f"Is there any missing value? {df0.isnull().values.any()}")
print(f"How many missing values: {df0.isnull().values.sum()}")

if "IPIP120" in ORIGINAL_IPIP_NEO_DATASET:
    print("Removing invalid data from dataset: IPIP120")
    df0 = df0[(df0.loc[:, "i1":"i120"] != 0).all(axis=1)]

df0 = df0.dropna(inplace=False)

print(f"New dataset total: {len(df0)}")
print(f"Number of countries: {len(df0.country.unique())}")

Dataset original total: 619150
Is there any missing value? False
How many missing values: 0
Removing invalid data from dataset: IPIP120
New dataset total: 410376
Number of countries: 241


## 3. Calculates scores 🧮

In [5]:
USER_SCORE = []

def compute_big5_score_120(r: pd.Series) -> bool | BaseException:
    item = []

    for x in range(1, 121):
        item.append(
            {
                "id_question": x,
                "id_select": r[f"i{x}"],
                "reverse_scored": 0,
            }
        )

    try:
        calc_score = IpipNeo(question=120, test=True).compute(
            sex="M" if int(r["sex"]) == 1 else "F",
            age=int(r["age"]),
            answers={"answers": item},
        )
        calc_score["case"] = r["case"]

        USER_SCORE.append(calc_score)
    except BaseException as e:
        raise BaseException(f"Failed: {e}")
    return True
    

In [6]:
assert len(USER_SCORE) == 0, "should be zero"
df0.apply(compute_big5_score_120, axis=1)

0         True
1         True
4         True
5         True
6         True
          ... 
619142    True
619143    True
619144    True
619147    True
619149    True
Length: 410376, dtype: bool

In [7]:
USER_SCORE_NORMALIZED = []

for x in USER_SCORE:
    try:
        USER_SCORE_NORMALIZED.append(
            T.extract_personalities(
                idx=x["case"], result=x["person"]["result"]["personalities"]
            )
        )
    except BaseException as e:
        raise BaseException(f"Failed: {e}")


In [8]:
df1 = pd.DataFrame(USER_SCORE_NORMALIZED)
df1_cols_numerics = df1.select_dtypes(include=["int", "float"]).columns

assert len(df0) == len(df1), "should be the same"

df2 = pd.merge(df0, df1, on="case", how="inner")
assert len(df2) == len(df0), "should be the same"

In [9]:
df2.head(10)

,case,sex,age,sec,min,...,facet_anger,facet_depression,facet_self_consciousness,facet_immoderation,facet_vulnerability
0,1,2,19,8,41,...,11.549711,58.018551,39.725121,96.864432,11.048211
1,2,2,22,24,45,...,89.843818,89.880570,57.486716,1.000000,80.327511
2,6,1,13,14,6,...,86.802916,29.000871,7.509259,6.890607,15.131043
3,7,2,18,25,11,...,57.719898,18.108659,8.870845,44.831762,5.285188
4,8,2,24,19,25,...,7.899414,26.475441,38.817649,62.697315,53.029251
5,9,2,20,26,30,...,73.862903,32.943919,58.257428,84.677109,46.924899
6,10,2,18,18,36,...,49.233759,11.912872,58.257428,55.597358,46.924899
7,12,1,50,27,48,...,4.344188,6.334921,3.908683,12.104661,6.911371
8,13,1,28,48,50,...,8.881105,22.370701,58.278921,12.269957,28.837680
9,16,2,19,19,20,...,2.841271,99.651096,30.798323,1.000000,11.048211


## 4. Save refined datasets ✅

In [10]:
assert len(df2) == 410376, "the count should be 410376!"

df2.to_csv(NQ_IPIP_NEO_DATASET, compression="gzip", index=False)

print(f"Dataset saved in: {NQ_IPIP_NEO_DATASET}")

Dataset saved in: ./datasets/refined/IPIP120-SCORES.csv.gz
